KAN THESIS

## Step 1: Import Libraries and Set Up the Environment
Starting with libs and setting up device. If possible use GPU

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from kan import KAN

# Set default dtype and device
torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


## Step 2: Load and Preprocess the MNIST Dataset
We're loading MNIST dataset and apply all transformation and normalization

In [2]:
#Loading MNIST dataset

transform = transforms.Compose([
    transforms.ToTensor(),  # PIL Image -> Tensor
    transforms.Normalize((0.1307,), (0.3081,)) #normalizing dataset 
    ]) 

#Download MNIST dataset and used the transform to convert the data to tensor and normalize it
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

#Create DataLoader to batch and shuffle the data using mulitprocessing
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:01<00:00, 7050919.44it/s] 


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 243421.31it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 2719383.02it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 1508952.77it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



## Step 3: Initialize the KAN Model
Here we create our KAN model with dimension for MNIST dataset.
Input set is only 784 pixels (images 28x28) and output size is 10
Parameters
Hidden layers -> More neurons may risk in overfitting
Grid -> More finner adjustment but more prone to overfitting
 

In [4]:
# Create a KAN model
#Input layer = 784, hidden layer1 = 128, hidden layer2 = 64, output layer = 10
#Grid = 5, k = 3, seed = 42
model = KAN(width=[784, 128, 64, 10], grid=5, k=3, seed=42, device=device)

# Move the model to the specified device
model = model.to(device)

checkpoint directory created: ./model
saving model version 0.0


## Step 4: Define the Loss Function and Optimizer
CrossEntropyLoss for classification and Adam optimizer for training model 

In [5]:
# Define loss function and optimizer // think about this params and below
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


## Step 5: Train the Model
This cell contains the training loop where the model is trained for a specified number of epochs. 
The loss is printed after each epoch to monitor the training process.

In [6]:
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.view(images.size(0), -1).to(device)  # Flatten the images
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')


KeyboardInterrupt: 

## Step 6: Evaluate the Model
Finally, we evaluate the trained model on the test set to see how well it performs. The accuracy is calculated and printed.


In [ ]:
# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(images.size(0), -1).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy on test set: {100 * correct / total:.2f}%')
